In [4]:
import torch
import deep.model as models
import deep.training as training
import pandas as pd
from tool.preprocessing import DataCollection
from tool.create_dataset import creation
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import numpy as np
import torch.nn as nn

import torch.optim as optim
import torch.utils.data as data
from sklearn.metrics import r2_score
from deep.training import training_mlp


#Config
PROB = 1
NUM_EPOCHS_ = 250
device= torch.device("cuda:0")

#Hyperparameters
NUM_RECORD = [6, 12, 20]
BATCH_SIZE=[ 64, 256, 512]
NUM_HIDDEN = [300, 500, 700]
NUM_LAYER = 6




#Data Loader
collection = DataCollection(drop_null=True)
gt = collection.get_gt()

X = torch.tensor([]).to(torch.device("cuda:0"))
y = torch.tensor([]).to(torch.device("cuda:0"))


In [5]:
#modello
#device

res_trainings = list()
try:
    for record in NUM_RECORD:
        for i in collection.get_devices():
            tmp = pd.merge(i,gt,how="inner",on="valid_at").rename(columns={"pm2p5_y":"pm2p5_t","pm2p5_x":"pm2p5"})
            res = creation(tmp,lookback=record,p=1)
            X = torch.concat([X.clone(),res[0].flatten(-2)])
            y = torch.concat((y.clone(),res[1].flatten(-2)[:,0]))

        for batch_s in BATCH_SIZE:
            for hidden in NUM_HIDDEN:
                for i in range(5):
                    
                    if i==0:
                        model = models.AirMLP_6(num_fin=record*6, num_hidden=hidden).to(device)
                    if i==1:
                        model = models.AirMLP_7(num_fin=record*6, num_hidden=hidden).to(device)
                    if i==2:
                        model = models.AirMLP_8(num_fin=record*6, num_hidden=hidden).to(device)
                    if i==3:
                        model = models.AirMLP_7h(num_fin=record*6, num_hidden=hidden).to(device)
                    if i==4:
                        model = models.AirMLP_8h(num_fin=record*6, num_hidden=hidden).to(device)

                    config = f"record: {record} total_dim: {record*6}, batch_size: {batch_s}, hidden: {hidden}, model: {model}"
                    print(config)
                    res_tmp = training_mlp(X,y,model,batch_s,NUM_EPOCHS_,device)
                    res_trainings.append((config,res_tmp))
except KeyboardInterrupt:
    pass

record: 6 total_dim: 36, batch_size: 64, hidden: 300, model: AirMLP_6(
  (net): Sequential(
    (0): BatchNorm1d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=36, out_features=300, bias=True)
    (2): ReLU()
    (3): Linear(in_features=300, out_features=300, bias=True)
    (4): ReLU()
    (5): Linear(in_features=300, out_features=300, bias=True)
    (6): ReLU()
    (7): Linear(in_features=300, out_features=300, bias=True)
    (8): ReLU()
    (9): Linear(in_features=300, out_features=300, bias=True)
    (10): ReLU()
    (11): Linear(in_features=300, out_features=1, bias=True)
  )
)
Epoch   0 train mse.: 15.319 test mse.: 15.320 || R-2 on test: -1.135
Epoch  10 train mse.: 6.127 test mse.: 6.315 || R-2 on test: 0.577
Epoch  20 train mse.: 5.966 test mse.: 6.192 || R-2 on test: 0.606
Epoch  30 train mse.: 5.474 test mse.: 5.744 || R-2 on test: 0.674
Epoch  40 train mse.: 5.149 test mse.: 5.425 || R-2 on test: 0.702
Epoch  50 train mse.: 5.